- manca da aggiungere il fit di tucci
- manca da vedere se il formato di compute summary gli piace o no
- manca il run per debagguare eventuali errori 

# Imported libraries and statistical functions

In [ ]:
import sys

sys.path.append("./../..")

from numba import jit
from numba.types import bool_, int_
import numpy as np
import matplotlib.pyplot as plt
from InternalLibrary.StatisticalFunctions import stat_corr_single, stat_s_redx
from scipy.optimize import curve_fit
from scipy.stats import moment

from tqdm import tqdm

from sbi import analysis as analysis
from sbi import utils as utils
from sbi.inference import SNPE, simulate_for_sbi, infer
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)
import torch
import statistics

from InternalLibrary.StatisticalFunctions import *
from InternalLibrary.SimulatorPackage import Simulator_noGPU
from pathos.multiprocessing import ProcessingPool as ProcessPool
import pathos

In [ ]:
# class of the simulator
class ParallelSimulator():
    def __init__(self, dt, DeltaT, TotalT, prior_lims, transient_time = 0, batch_size = 50, multi_round_mode = False, prior = None) -> None:
        self.sim_param = {'dt': dt, 'DeltaT': DeltaT, 'TotalT': TotalT, 'transient_time': transient_time}
        self.pool = ProcessPool(nodes=pathos.helpers.cpu_count())
        self.batch_size = batch_size
        self.prior_limits = prior_lims
        _ = self._dummy_run()
        self.multi_round_mode = multi_round_mode
        self.prior = prior
    
    def _dummy_run(self):
        Simulator_noGPU(dt = 1, DeltaT=1, TotalT = 2, theta = np.zeros(5).reshape(5,1,1), transient_time = 0)
    
    def _run_simulator(self, theta):
        return Simulator_noGPU(theta = theta, **self.sim_param)
    
    def _get_theta(self):
        if self.multi_round_mode:
            return self.prior.sample((self.batch_size,1)).T.reshape(5,self.batch_size,1).numpy()
        return get_theta_from_prior(self.prior_limits, self.batch_size)[0]
    
    def run(self,N):
        thetas = [self._get_theta() for _ in range(N)]
        s = self.pool.map(self._run_simulator, thetas)
        x = s[0][0]
        y = s[0][1]
        f = s[0][2]
        for i in np.arange(1,len(s)):
            x = np.concatenate([x, s[i][0]])
            y = np.concatenate([y, s[i][1]])
            f = np.concatenate([f, s[i][2]])
        t = thetas[0]
        for i in np.arange(1,len(thetas)):
            t = np.concatenate([t, thetas[i]],axis = 1)
        return x, y, f, t
    
    def same_theta_test(self,N,theta):
        thetas = [theta for _ in range(N)]
        s = self.pool.map(self._run_simulator, thetas)
        x = s[0][0]
        y = s[0][1]
        f = s[0][2]
        for i in np.arange(1,len(s)):
            x = np.concatenate([x, s[i][0]])
            y = np.concatenate([y, s[i][1]])
            f = np.concatenate([f, s[i][2]])
        return x, y, f
    
    
    
def prior_from_lims(prior_lims):
    NumberOfParameters = len(prior_lims)
    BoxLimits = np.zeros((2, NumberOfParameters))
    i = 0
    for item in prior_lims.items():
        BoxLimits[0, i] = item[1][0]
        BoxLimits[1, i] = item[1][1]
        i +=1
    BoxLimits = torch.tensor(BoxLimits)
    prior = utils.BoxUniform(low=BoxLimits[0], high=BoxLimits[1])
    return prior

In [ ]:
## functions for the fit 
def stat_fit_corr(single_corr, DeltaT):
    """
    Fit the correlation function with a sum of 3 exponentials
    """
    t_cxx = np.linspace(0., (len(single_corr)+1)*DeltaT, (len(single_corr)+1))[1:]
    
    def cxx_exp3(t, a1, a2, a3, b1, b2, b3):
        return a1*np.exp(-b1*t) + a2*np.exp(-b2*t) + a3*np.exp(-b3*t)
    
    try: 
        popt, pcov = curve_fit(cxx_exp3, t_cxx, single_corr, p0=[1e2, 1e2, 1e2, 10, 10, 10], maxfev=5000)
    except:
        return np.zeros(6)

    return popt, cxx_exp3(t_cxx, *popt)

def stat_fit_s_redx(single_s_redx, DeltaT, mode="exp"):
    """
    Fit the s_redx function
    """
    assert mode in ["exp", "simple"], "Mode not recognized"

    t_cxx = np.linspace(0., (len(single_s_redx)+1)*DeltaT, (len(single_s_redx)+1))[1:]
    
    def s_redx_simple(t, a, tau):
        return(1 + a*t/(1+t/tau))

    def s_redx_exp(t, a1, a2, b1, b2, b3, c):
        a3 = 1 - a1 - a2 
        sum_exp = a1*np.exp(-b1*t) + (a2)*np.exp(-(b1+b2)*t) + (a3)*np.exp(-(b1+b2+b3)*t)
        sum = a1*b1 + (a2)*(b1+b2) + (a3)*(b1+b2+b3)
        tau = 1/sum
        return(1 + c - (c*tau/t)*(1-sum_exp))
    
    if mode == "exp":
        try: 
            popt, pcov = curve_fit(s_redx_exp, t_cxx, single_s_redx, p0=[1, 10, 10, 0.1, 0.01, 10],
                          bounds=([0, 0, 0, 0, 0, 0], [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]), maxfev=5000)
        except:
            return np.zeros(6)
        return popt, s_redx_exp(t_cxx, *popt)

    if mode == "simple":
        try:
            popt, pcov = curve_fit(s_redx_simple, t_cxx, single_s_redx, p0=[1e3, 1e-2],
                          bounds=([0, 0], [np.inf, np.inf]), maxfev=5000)
        except:
            return np.zeros(6)
        return popt, s_redx_simple(t_cxx, *popt)
        

def select_summary_statistics(summary_statistics, selected_statistics, DeltaT,
                              z_score=False, cl_lin=-1, cl_log=-1, fit_cxx=False, fit_s_redx=False):
    selected_statistics = selected_statistics.copy()
    
    # Check that the selected statistics are in the summary statistics
    assert set(selected_statistics).issubset(set(summary_statistics.keys()))
    "The selected statistics are not in the summary statistics"

    # Checks on postprocessing
    assert cl_log < 0 or cl_lin < 0, "You cannot subsample bot 'lin' and 'log' at the same time"
    if cl_lin > 0 or cl_log > 0: assert (fit_cxx == False) and (fit_s_redx == False), "You cannot subsample and fit at the same time"
    
    # Post-subselection of Cxx and s_redx
    if cl_lin > 0:
        idx_clean_corr = np.linspace(0, len(summary_statistics["Cxx"])-1, cl_lin, dtype=np.int32)
        if "Cxx" in selected_statistics:
            summary_statistics["Cxx"] = summary_statistics["Cxx"][idx_clean_corr]
        if "s_redx" in selected_statistics:
            summary_statistics["s_redx"] = summary_statistics["s_redx"][idx_clean_corr]
        if "s_red1" in selected_statistics:
            summary_statistics["s_red1"] = summary_statistics["s_red1"][idx_clean_corr]
        if "s_red2" in selected_statistics:
            summary_statistics["s_red2"] = summary_statistics["s_red2"][idx_clean_corr]

    if cl_log > 0:
        idx_clean_corr = np.logspace(0, np.log10(len(summary_statistics["Cxx"])-1), cl_log, dtype=np.int32)
        if "Cxx" in selected_statistics:
            summary_statistics["Cxx"] = summary_statistics["Cxx"][idx_clean_corr]
        if "s_redx" in selected_statistics:
            summary_statistics["s_redx"] = summary_statistics["s_redx"][idx_clean_corr]
        if "s_red1" in selected_statistics:
            summary_statistics["s_red1"] = summary_statistics["s_red1"][idx_clean_corr]
        if "s_red2" in selected_statistics:
            summary_statistics["s_red2"] = summary_statistics["s_red2"][idx_clean_corr]

    # Fit Cxx and s_redx and use the paramaters as summary statistics
    if fit_cxx and "Cxx" in selected_statistics:
        summary_statistics["Cxx"] = stat_fit_corr(summary_statistics["Cxx"], DeltaT)[0]
        if (summary_statistics["Cxx"] == np.zeros(6)).all(): return None

    if fit_s_redx and "s_redx" in selected_statistics:
        summary_statistics["s_redx"] = stat_fit_s_redx(summary_statistics["s_redx"], DeltaT, mode=fit_s_redx)[0]
        if (summary_statistics["s_redx"] == np.zeros(6)).all(): return None
        
    # Check if theta is selected for testing reasons
    theta_selected = False
    if "theta" in selected_statistics:
        theta_selected = True
        selected_statistics.remove("theta")

    # Get the selected summary statistics in a torch tensor
    if z_score:
        list_of_statistics = [torch.tensor(zscore(summary_statistics[i])) for i in selected_statistics]
    else:   
        list_of_statistics = [torch.tensor(summary_statistics[i]) for i in selected_statistics]
    selected_summary_statistics = torch.cat(list_of_statistics, dim=0)
    selected_summary_statistics = torch.unsqueeze(selected_summary_statistics, 0)

    # Add theta to the summary statistics if selected
    if theta_selected:
        theta = torch.tensor(summary_statistics["theta"])
        selected_summary_statistics = torch.cat((selected_summary_statistics, theta.T), dim=1)

    # Convert the summary statistics to float32 (required for sbi)
    selected_summary_statistics = selected_summary_statistics.to(torch.float32)
    return selected_summary_statistics

In [ ]:
# function to compute the summary statistics and the fit of them
def Computesummary(multiple_raw_x_traces,TotalT,transient = 0.5,DeltaT = 1/25_000, function = None):
    t = np.linspace(transient, TotalT, multiple_raw_x_traces[0].shape[0])
    autocorr = stat_corr_single(multiple_raw_x_traces[0,:], 1) .reshape(1,-1)
    n_sim = multiple_raw_x_traces.shape[0]
    if n_sim != 1:
        for i in np.arange(1,n_sim):
            autocorr = np.vstack((autocorr, stat_corr_single(multiple_raw_x_traces[i,:], DeltaT)))
    
    t_corr = TotalT/10
    _,_,RSVP = stat_s_redx(autocorr[0], t_corr, t)
    if n_sim != 1:
        for i in np.arange(1,n_sim):
            _,_,RSVP_ = stat_s_redx(autocorr[i], t_corr, t)
            RSVP = np.vstack((RSVP, RSVP_))
        
    
    ### insert the function for tucci's fit here ###
    
    #fit the summary staristics
    summary_stats = {'RSVP': RSVP, 'autocorr': autocorr}
    fitted_summary_stats = select_summary_statistics(summary_statistics=summary_stats, selected_statistics=["RSVP", 'autocorr'], DeltaT=DeltaT, z_score=False, cl_lin=-1, cl_log=5, fit_cxx=True, fit_s_redx=True)
    
    return fitted_summary_stats, summary_stats


# Algorithm

In [ ]:
# Set Up Parameters
dt = 1e-6
DeltaT = 1/25_000
TotalT = 10.
transient = 0.5
prior_limits = {
    "mu_y": [1e4, 140e4],
    "k_y": [1.5e-2, 30e-2],
    "k_int": [1e-3, 6e-3],
    "tau": [2e-2, 20e-2],
    "eps": [0.5, 6],
}

batch_size = 200
N = pathos.helpers.cpu_count()
TotalSim = batch_size * 16

In [ ]:
def MultiRoundPipeline(dt,DeltaT,TotalT,prior_lims,transient,batch_size,N,n_rounds):
    Simulator = ParallelSimulator(dt, DeltaT, TotalT, prior_lims, transient_time = transient,batch_size = 1)
    x, y, f, thetas_exp = Simulator.run(1)
    summmary_fit_exp,_ = Computesummary(x, TotalT)
    
    NumberOfParameters = len(prior_limits)
    BoxLimits = np.zeros((2, NumberOfParameters))
    i = 0
    for item in prior_limits.items():
        BoxLimits[0, i] = item[1][0]
        BoxLimits[1, i] = item[1][1]
        i +=1
    BoxLimits = torch.tensor(BoxLimits)
    prior = utils.BoxUniform(low=BoxLimits[0], high=BoxLimits[1])
    prior, _, _ = process_prior(prior)
    infer = SNPE(prior=prior)
    proposal = prior
    posteriors = []
    for _ in np.arange(n_rounds):
        print(f"\nRound: {_+1} / {n_rounds}")
        Simulator = ParallelSimulator(dt, DeltaT, TotalT, prior_lims, transient_time = transient,batch_size = batch_size, multi_round_mode=True, prior = proposal)
        x, y, f, thetas = Simulator.run(N)
        summmary_fit = Computesummary(x, TotalT)
        summmary_fit = torch.tensor(summmary_fit).float()
        theta = torch.tensor(thetas).float().T.reshape(N*batch_size,5)
        density_estimator = infer.append_simulations(theta, summmary_fit, proposal = proposal).train()
        posterior = infer.build_posterior(density_estimator)
        posterior = posterior.set_default_x(summmary_fit_exp)
        posteriors.append(posterior)
        proposal = posterior
    return posteriors, thetas_exp

In [ ]:
pos, thetas_exp = MultiRoundPipeline(dt,DeltaT,TotalT,prior_limits,transient,batch_size,N,2);

In [ ]:
last_posterior = pos[-1]
samples = last_posterior.sample((10000,))
fig, ax = analysis.pairplot(samples, figsize=(10, 6), points=thetas_exp.reshape(5,));